# Exploring the Data

In [ ]:
###########################################
# Suppress matplotlib user warnings
# Necessary for newer version of matplotlib
import warnings
warnings.filterwarnings("ignore", category = UserWarning, module = "matplotlib")
#
# Display inline matplotlib plots with IPython
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
###########################################

# Import all of the libraries necessary for this project
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBRegressor
#from xgboost import cv
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Import visualization libraries
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

# Success - Display the first 5 records
display(train_data.head(n=5))
# How many records and columns do we have?
print("This dataset contains %d observations with %d columns for each." % train_data.shape)

# Features

1. **SalePrice:** the property's sale price in dollars. This is the target variable that you're trying to predict.
1. **MSSubClass:** The building class
1. **MSZoning:** The general zoning classification
1. **LotFrontage:** Linear feet of street connected to property
1. **LotArea:** Lot size in square feet
1. **Street:** Type of road access
1. **Alley:** Type of alley access
1. **LotShape:** General shape of property
1. **LandContour:** Flatness of the property
1. **Utilities:** Type of utilities available
1. **LotConfig:** Lot configuration
1. **LandSlope:** Slope of property
1. **Neighborhood:** Physical locations within Ames city limits
1. **Condition1:** Proximity to main road or railroad
1. **Condition2:** Proximity to main road or railroad (if a second is present)
1. **BldgType:** Type of dwelling
1. **HouseStyle:** Style of dwelling
1. **OverallQual:** Overall material and finish quality
1. **OverallCond:** Overall condition rating
1. **YearBuilt:** Original construction date
1. **YearRemodAdd:** Remodel date
1. **RoofStyle:** Type of roof
1. **RoofMatl:** Roof material
1. **Exterior1st:** Exterior covering on house
1. **Exterior2nd:** Exterior covering on house (if more than one material)
1. **MasVnrType:** Masonry veneer type
1. **MasVnrArea:** Masonry veneer area in square feet
1. **ExterQual:** Exterior material quality
1. **ExterCond:** Present condition of the material on the exterior
1. **Foundation:** Type of foundation
1. **BsmtQual:** Height of the basement
1. **BsmtCond:** General condition of the basement
1. **BsmtExposure:** Walkout or garden level basement walls
1. **BsmtFinType1:** Quality of basement finished area
1. **BsmtFinSF1:** Type 1 finished square feet
1. **BsmtFinType2:** Quality of second finished area (if present)
1. **BsmtFinSF2:** Type 2 finished square feet
1. **BsmtUnfSF:** Unfinished square feet of basement area
1. **TotalBsmtSF:** Total square feet of basement area
1. **Heating:** Type of heating
1. **HeatingQC:** Heating quality and condition
1. **CentralAir:** Central air conditioning
1. **Electrical:** Electrical system
1. **1stFlrSF:** First Floor square feet
1. **2ndFlrSF:** Second floor square feet
1. **LowQualFinSF:** Low quality finished square feet (all floors)
1. **GrLivArea:** Above grade (ground) living area square feet
1. **BsmtFullBath:** Basement full bathrooms
1. **BsmtHalfBath:** Basement half bathrooms
1. **FullBath:** Full bathrooms above grade
1. **HalfBath:** Half baths above grade
1. **Bedroom:** Number of bedrooms above basement level
1. **Kitchen:** Number of kitchens
1. **KitchenQual:** Kitchen quality
1. **TotRmsAbvGrd:** Total rooms above grade (does not include bathrooms)
1. **Functional:** Home functionality rating
1. **Fireplaces:** Number of fireplaces
1. **FireplaceQu:** Fireplace quality
1. **GarageType:** Garage location
1. **GarageYrBlt:** Year garage was built
1. **GarageFinish:** Interior finish of the garage
1. **GarageCars:** Size of garage in car capacity
1. **GarageArea:** Size of garage in square feet
1. **GarageQual:** Garage quality
1. **GarageCond:** Garage condition
1. **PavedDrive:** Paved driveway
1. **WoodDeckSF:** Wood deck area in square feet
1. **OpenPorchSF:** Open porch area in square feet
1. **EnclosedPorch:** Enclosed porch area in square feet
1. **3SsnPorch:** Three season porch area in square feet
1. **ScreenPorch:** Screen porch area in square feet
1. **PoolArea:** Pool area in square feet
1. **PoolQC:** Pool quality
1. **Fence:** Fence quality
1. **MiscFeature:** Miscellaneous feature not covered in other categories
1. **MiscVal:** $Value of miscellaneous feature
1. **MoSold:** Month Sold
1. **YrSold:** Year Sold
1. **SaleType:** Type of sale
1. **SaleCondition:** Condition of sale

In [ ]:
# Let's quickly see if there are any correlations that stand out
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(train_data.corr(), annot=True, linewidths=.25, fmt= '.1f',ax=ax)
plt.show()

In our correlation heatmap we are looking for values that highly correlate with `SalePrice` because we want to make sure there isn't a case of data leakage. We can see that `OverallQual` has the highest correlation at ~0.8 which makes sense because the quality of the house should correlate with its price.

In [ ]:
# Plot columns with missing values to evaluate how many are missing
train_data.isnull().sum().plot(kind='bar', figsize=(15, 10))
plt.show()

From our bar chart we observe that there are mostly missing values in columns `Alley`, `PoolQC`, `Fence` and `MiscFeature`. We will test removing these and imputing with flagging to see which method returns the best results. Sometimes knowing a feature is missing can be valuable.

In [ ]:
# Let's have a look at some statistics
display(train_data.describe())
#display(train_data.columns)

We can see that there are some outliers in our data which are significantly greater than a majority of the dataset. These might skew our analysis. Let's try seeing how these features are distributed.

In [ ]:
def distribution(data, features = [], transformed = False):
    """
    Visualization code for displaying distributions of features
    """
    # Calculate number of rows needed for plots
    if len(features) % 2 == 1:
        rows = (len(features)-1)/2
    elif len(features)==0:
        data = data.dropna(axis=1)
        features = data.columns
        if len(features) % 2 == 1:
            rows = (len(features)-1)/2
        else:
            rows = len(features)/2
    else:
        rows = len(features)/2

    # Create figure
    fig = plt.figure(figsize = (11,5*rows))

    # Skewed feature plotting
    for i, feature in enumerate(features):
        ax = fig.add_subplot(rows, 2, i+1)
        ax.hist(data[feature], bins = 25, color = '#00A0A0')
        ax.set_title("'%s' Feature Distribution"%(feature), fontsize = 14)
        ax.set_xlabel("Value")
        ax.set_ylabel("Number of Records")
        #ax.set_ylim((0, 2000))
        #ax.set_yticks([0, 500, 1000, 1500, 2000])
        #ax.set_yticklabels([0, 500, 1000, 1500, ">2000"])

    # Plot aesthetics
    if transformed:
        fig.suptitle("Transformed Distributions of Data Features", \
            fontsize = 16, y = 1.03)
    else:
        fig.suptitle("Distributions of Data Features", \
            fontsize = 16, y = 1.03)

    fig.tight_layout()
    fig.show()

In [ ]:
#distribution(train_data)

In [ ]:
# TODO: apply a scaling function to skewed features

## Preparing the Data

We are going to compare a few preprocessing methods below and choose the one that gives us the best results. Best results here will be defined as the model that returns the lowest MAE.

In [ ]:
y = train_data.SalePrice
X_one_hot_encoded = pd.get_dummies(train_data).drop('SalePrice', axis=1)

# Creating our pipelines to test different preprocessing techniques
pipeline_one_hot_encoded = Pipeline([('imputer', Imputer()),
                                     ('xgbrg', XGBRegressor())])

# Comparing preprocessing techniques with cross validation
mae_one_hot_encoded = cross_val_score(pipeline_one_hot_encoded, X_one_hot_encoded, y, scoring='neg_mean_absolute_error')
print('CV Mean Absolute Error with One-Hot Encoding: %2f' %(-1 * mae_one_hot_encoded.mean()))

# Using grid search cv
from sklearn.model_selection import GridSearchCV

pipeline_grid_search_cv = Pipeline([('imputer', Imputer()),
                                    ('xgbrg', XGBRegressor())])
params = {"xgbrg__n_estimators": [25, 50, 100, 150, 200, 300],
         "xgbrg__learning_rate": [0.001, 0.01, 0.1, 1.0]} 
searchCV = GridSearchCV(pipeline_grid_search_cv, params, n_jobs=1, cv=5, refit=True)
searchCV.fit(X_one_hot_encoded, y)
final_pipeline = searchCV.best_estimator_

#15917.110161
mae_grid_search_cv = cross_val_score(final_pipeline, X_one_hot_encoded, y, scoring='neg_mean_absolute_error')
print('CV Mean Absolute Error with GridSearchCV: %2f' %(-1 * mae_grid_search_cv.mean()))

In [ ]:
# TODO: Partial Dependence Plots

# Prediction

In [ ]:
import copy
# Read the test data
ID = test_data.Id

# Join the data and then do one hot encoding to get the same amount of features, then split again
train_objs_num = len(train_data)
dataset = pd.concat(objs=[train_data, test_data], axis=0)
dataset = pd.get_dummies(dataset)
train_data = copy.copy(dataset[:train_objs_num])
test_data = copy.copy(dataset[train_objs_num:])
test_data = test_data.drop('SalePrice', axis=1)

#X_one_hot_encoded = train_data.drop('SalePrice', axis=1)
#train_X_ohe,test_X_ohe,train_y_ohe,test_y_ohe = train_test_split(X_one_hot_encoded, y, test_size=0.25, random_state=1)
#pipeline_grid_search_cv = make_pipeline(Imputer(),
#                    GridSearchCV(XGBRegressor(),
#                                 param_grid={'n_estimators': [50, 150]},
#                                 cv=2,
#                                 refit=True))
#pipeline_grid_search_cv.fit(X_one_hot_encoded, train_data.SalePrice)

# Use the model to make predictions
predicted_prices = final_pipeline.predict(test_data)

# We will look at the predicted prices to ensure we have something sensible.
print(predicted_prices)

In [ ]:
my_submission = pd.DataFrame({'Id': ID, 'SalePrice': predicted_prices})

# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)
